#### 导入工具包

In [11]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from math import log
from collections import Counter

#### 创建数据集

In [12]:
def create_data():
    # 创建数据集
    datasets = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ]
    # 给出数据集的列名称
    labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
    
    return datasets, labels

In [13]:
datasets, labels = create_data()
train_data = pd.DataFrame(datasets, columns=labels)

#### 计算信息增益

#####  计算基尼系数

In [17]:
def calc_jini(dataframe, c=-1):
    target = dataframe.iloc[:, c]
    data_length = len(target)
    labels = Counter(target)
    ent = 1-sum([np.power((p/data_length), 2) for p in list(labels.values())] )
    return ent

In [10]:
calc_ent(train_data, -1)

0.48

##### 计算特征拆分之后的基尼系数

In [18]:
def cond_jini(dataframe, index=0):
    data_length = len(dataframe)
    feature_set = {}
    data_group = dataframe.groupby(dataframe.iloc[:, index])
    for group in data_group:
        key = group[0]
        val = group[1]
        feature_set[key] = val
    
    cond_jini = sum([(len(p)/data_length)*calc_jini(p, -1) for p in feature_set.values()])
    return cond_jini

In [19]:
cond_jini(train_data, 1)

0.31999999999999995

##### 计算基尼系数的增益

In [24]:
def info_jini(jini, cond_jini):
    return jini - cond_jini

##### 计算基尼系数增益最大的特征

In [30]:
def info_gain_jini(dataframe):
    count = dataframe.shape[1] - 1
    ent = calc_ent(dataframe)
    best_feature = []
    for c in range(count):
        c_info_jini = info_jini(calc_jini(dataframe, -1), cond_jini(dataframe, index=c))
        best_feature.append((c, c_info_jini))
        print('特征({}) - info_jini - {:.3f}'.format(labels[c], c_info_jini))
    best_ = max(best_feature, key=lambda x: x[-1])
    return '特征({})的基尼增益最大，选择为根节点特征'.format(labels[best_[0]])

In [31]:
info_gain_jini(train_data)

特征(年龄) - info_jini - 0.053
特征(有工作) - info_jini - 0.160
特征(有自己的房子) - info_jini - 0.213
特征(信贷情况) - info_jini - 0.196


'特征(有自己的房子)的基尼增益最大，选择为根节点特征'

#### 使用类组织代码

##### 构造树结构

In [32]:
class Node:
    def __init__(self, root=True, label=None, feature_name=None, feature=None):
        # 节点处理思路
        """
        1. 对于叶子节点，返回该节点对应的所有的样本标签值
        2. 对于中间节点，返回各个特征拆分后对应的子节，并添加到树结构中
        """
        self.root = root               # 当前节点是否是叶子节点
        self.label = label             # 如果当前节点是是叶子节点，则返回该节点对应的预测值
        self.feature_name = feature_name        # 用作切分的特征的名称
        self.feature = feature                  # 用作切分的特征对应的索引值
        self.tree = {}                          # 树结构
        self.result = {
            'label': self.label,
            'feature': self.feature,
            'tree': self.tree
        }
        
    def __repr__(self):
        return '{result}'.format(result = self.result)
    
    # 训练过程中使用
    def add_node(self, val, node):
        self.tree[val] = node
    
    # 预测过程中使用
    def predict(self, features):
        if self.root is True:
            return self.label
        return self.tree[features[self.feature]].predict(features)

##### 填充树结构

In [39]:
class DTree:
    def __init__(self, epsilon=0.1):
        self.epsilon = epsilon           # 超参数，当信息增益小于某个值的时候不再继续分裂
        self._tree = {}
        
    # 计算基尼系数
    @staticmethod
    def calc_jini(dataframe, c=-1):
        target = dataframe.iloc[:, c]
        data_length = len(target)
        labels = Counter(target)
        ent = 1-sum([np.power((p/data_length), 2) for p in list(labels.values())] )
        return ent
    
    # 计算拆分后特征的基尼系数
    def cond_jini(self, dataframe, index=0):
        data_length = len(dataframe)
        feature_set = {}
        data_group = dataframe.groupby(dataframe.iloc[:, index])
        for group in data_group:
            key = group[0]
            val = group[1]
            feature_set[key] = val

        cond_jini = sum([(len(p)/data_length)*self.calc_jini(p, -1) for p in feature_set.values()])
        return cond_jini
    
    # 计算基尼增益
    @staticmethod
    def info_jini(jini, cond_jini):
        return jini - cond_jini
    
    # 返回基尼增益最大的特征
    def info_gain_jini(self, dataframe):
        count = dataframe.shape[1] - 1
        ent = calc_ent(dataframe)
        best_feature = []
        for c in range(count):
            c_info_jini = self.info_jini(self.calc_jini(dataframe, -1), self.cond_jini(dataframe, index=c))
            best_feature.append((c, c_info_jini))
        best_ = max(best_feature, key=lambda x: x[-1])
        return best_

    # 定义训练部分
    def train(self, train_data):
        y_train, features = train_data.iloc[:, -1], list(train_data.columns)[:-1]
        
        """设置边界条件"""
        # 1.如果节点中的标签属于同一类别，则当前节点为叶子节点
        if len(y_train.value_counts()) == 1:
            return Node(root=True, label=y_train.iloc[0])
        
        # 2.如果拆分到最后，没有可以继续拆分的特征了，则将当前节点中数量最大的的类别作为最终的类别
        if len(features) == 0:
            return Node(root=True, label=y_train.value_counts().sort_values(ascending=False).index[0])
        
        # 3. 如果当前节点继续拆分之后的基尼增益小于阈值，则停止拆分
        max_feature, max_info_gain = self.info_gain_jini(train_data)
        max_feature_name = features[max_feature]
        if max_info_gain < self.epsilon:
            return Node(root=True, label=y_train.value_counts().sort_values(ascending=False).index[0])
        
        """如果当前节点不满足成为叶子节点的条件，则成为分枝节点，继续拆分"""
        node_tree = Node(root=False, feature_name=max_feature_name, feature=max_feature)
        feature_list = train_data[max_feature_name].value_counts().index
        # 4. 递归生成树
        for f in feature_list:
            sub_train_df = train_data.loc[train_data[max_feature_name]==f].drop(max_feature_name, axis=1)
            sub_tree = self.train(sub_train_df)
            node_tree.add_node(f, sub_tree)
        
        return node_tree
    
    def fit(self, train_data):
        self._tree = self.train(train_data)
        return self._tree
    
    def predict(self, X_test):
        return self._tree.predict(X_test)

##### 运行并查看结果

In [40]:
# 创建数据集
datasets, labels = create_data()
data_df = pd.DataFrame(datasets, columns=labels)

# 创建树
dt = DTree()
tree = dt.fit(data_df)

# 查看树结构
print(tree)

# 预测数据
dt.predict(['老年', '否', '否', '一般'])

{'label': None, 'feature': 2, 'tree': {'否': {'label': None, 'feature': 1, 'tree': {'否': {'label': '否', 'feature': None, 'tree': {}}, '是': {'label': '是', 'feature': None, 'tree': {}}}}, '是': {'label': '是', 'feature': None, 'tree': {}}}}


'否'